# 课程内容

本课利用百度人脸识别应用、百度语音合成技术进行人脸识别和语音播报。

本课引入多任务的概念，利用我们的多核处理器，让树莓派动起来，让显示更流畅。

百度百科: 多任务处理是指系统可同时运行多个进程，而每个进程也可同时执行多个线程。一个线程是指程序的一条执行路径，它在系统指定的时间片中完成特定的功能。系统不停地在多个线程之间切换，由于时间很短，看上去多个线程在同时运行。或者对于在线程序可并行执行同时服务于多个用户称为多任务处理。

# 开始我们的实验

## (1)认识多任务
我们之前的程序为

    if __name__ == "__main__":
        try:
            while True:
                程序主体

        except KeyboardInterrupt:  
            程序退出处理

我们的程序一直在"while True"里不断地执行，对于简单任务来说，这样的程序能满足需要，但对于稍微复杂点的程序，这样的设计就不能满足我们的需求了。

例如:上节课的程序，我们获取图像，发送百度AI进行分析，然后显示处理结果；在进行分析和显示处理结果时，读取图像的功能就没有执行。

我们想要的效果就是，在读取图像进行显示的同时，进行图像进行百度AI分析，同时进行其他的一些操作(例如，本课的语音提示戴口罩)。如何来实现我们的想法呢?

我们需要引入多任务。

python的多任务库为:from multiprocessing import Process, Queue
其中Process为多任务的模块，Queue为多任务通信的消息队列机制。

如何使用多任务？就两步。

①建立多任务。

    建立一个多任务
    get_camera_frame = Process(target=camera_frame_func, args=("获取摄像头图像", q_frame, mydict))
    参数:target为该任务的处理函数
    参数:args为处理函数的参数，"获取摄像头图像"为该任务的名称；q_frame为传递给该任务的第一个参数；mydict为传递给该任务的第二个参数。
    target为该任务的处理函数的格式:
    #　获取摄像头图像
    def camera_frame_func(task_name, mult_queue1, mydict):  
        #　给出提示信息
        
        print(task_name + "任务启动")
        try:
            while True:
                #添加自己的函数处理模块
  
        except KeyboardInterrupt:
            #程序退出处理
     
    返回值:get_camera_frame为该任务的处理对象。   
    发现没有，处理函数的格式就是我们之前的程序。

②启动该任务
    
    #启动get_camera_frame任务
    get_camera_frame.start()
    只需要调用get_camera_frame对象的start方法就可以启动该任务。
    
这样启动了多任后，例如我们建立了4个任务，操作系统会按照相应的调度策略让任务进行运行，对于我们所使用的树莓派4B平台，有4个处理器核，每个核上可以运行一个任务，操作系统会调度合适的任务到一个核上处理，这样就做到的真正的同时运行程序的效果。

## (２)任务间通信(进程间通信)
我们的启动后，都在各自执行各自的工作，但他们没有交互，我该如何将任务1读取到的图像传递给任务2进行百度AI分析呢?
这时，我们需要引入任务间通信，本次我们使用消息队列的方式。

python的多任务库为:from multiprocessing import Process, Queue
其中Process为多任务的模块，Queue为多任务通信的消息队列机制。

我们该如何使用消息队列呢?

如何使用多任务？就三步。

①建立一个消息队列

    q_frame = Queue()
    q_frame就是一个消息队列的对象，我们使用这个他就行了。
    
②传递消息队列到进程

    Process(target=camera_frame_func, args=("获取摄像头图像", q_frame))
    在进程参数中添加该消息队列，就可以将消息队列传递给该进程
    
③向消息队列发送消息或者从消息队列读取消息

    向消息队列发送数据:
    mult_queue1.put(frame) 
    调用消息队列的put方法，就将frame对象的数据放到消息队列中
    
    从消息队列读取数据:
    frame = mult_queue.get()
    调用消息队列的get方法，就将取出的对象放在frame中（准确的说是frame指向该对象）。
    

In [ ]:
######################################################
#
# 先感受下小派的口罩检测，带好口罩，挡住病毒,选择本cell，按shirt+enter键运行本模块
#
######################################################
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#
# Copyright (c) 2020 Taste all Pi.
#
# Licensed under the GNU General Public License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#   http://www.gnu.org/licenses/gpl-2.0.html
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#导入标准库
import sys
import os
from playsound import playsound
import cv2 as cv
import time
from multiprocessing import Process, Queue
import multiprocessing

#导入自定义库
sys.path.append('../baidu_api_lib')
from baidu_picture import baidu_picture_2_msg
from baidu_sound import baidu_word_2_sound

""" 公开课语音合成+人脸识别，可选更改为自己的api接口 """
APP_ID = '21139072'
API_KEY = 'tZ0j3YspNiGDhLeO7vxxvdzb'
SECRET_KEY = 'UaaaMyO8RiZvXj0HxEt9DQavuk6u2uA3'

#百度AI的调用url
baidu_request_url = "https://aip.baidubce.com/rest/2.0/face/v3/detect"

#处理图像的频率
FRAME_PROC_FR = 80

#　获取摄像头图像
def camera_frame_func(task_name, mult_queue1, mydict):
    
    # 创建一个VideoCapture对象
    capture = cv.VideoCapture(0)   
    
    #　给出提示信息
    print(task_name + "任务启动")
    
    #　传递视频的周期
    pic_hz = 0
    
    try:
        while True:
            # 一帧一帧读取视频
            ret, frame = capture.read()
            
            #　将图像发送至队列中
            pic_hz+=1
            if (pic_hz % FRAME_PROC_FR) == 1:     
                mult_queue1.put(frame) 
            
            #获取人脸位置
            if mydict:   
                #获取脸的个数
                face_num = mydict["result"]["face_num"]
                
                #有人脸才进行画框
                if face_num:
                    #获取人脸位置，画出图框
                    for i in range(face_num):
                        location = mydict["result"]["face_list"][i]["location"]
                        left_top = (int(location['left']), int(location['top']))
                        right_bottom = (int(left_top[0] + location['width']), int(left_top[1] + location['height']))
                        cv.rectangle(frame, left_top, right_bottom, (0,255,0),2) 
            
            # 本地显示视频图像
            cv.imshow('capture', frame) 
            cv.waitKey(1)

    except KeyboardInterrupt:
        # 释放cap,销毁窗口
        capture.release()      
        print(task_name + "任务被终止")
        
#处理图像
def proc_frame_func(task_name, mult_queue, mult_queue2, mydict):
    #　给出提示信息
    print(task_name + "任务启动")
    
    # 传入百度AI的参数
    pic_msg = baidu_picture_2_msg(APP_ID, API_KEY, SECRET_KEY)
    
    try:
        while True:
            #从队列中获取图片
            frame = mult_queue.get()
            
            # 写入图片
            cv.imwrite('camera_pic.jpg',frame) 
            
            #从百度AI获取图片分析结果
            response = pic_msg.pic_2_msg(baidu_request_url, 'camera_pic.jpg')
            
            #获取分析数据是否正确
            error_msg = response.json()["error_msg"]
            
            #获取到正确信息
            if error_msg == 'SUCCESS': 
                #更新人脸识别信息
                mydict["result"] = response.json()["result"]
            
            #将播报信息进行对外发送
            mult_queue2.put(response) 
            
    except KeyboardInterrupt:
        os.remove('camera_pic.jpg')
        print(task_name + "任务被终止")
        
#播报语音信息
def read_rst_func(task_name, mult_queue):
    print(task_name + "任务启动")
    
    #传入百度AI的参数
    word_2_sound = baidu_word_2_sound(APP_ID, API_KEY, SECRET_KEY)
    
    try:
        while True:
            #从队列中获取播报信息
            response = mult_queue.get()
            
            #获取分析数据是否正确
            error_msg = response.json()["error_msg"]
            
            #获取到正确信息
            if error_msg == 'SUCCESS':   
                #获取脸的个数
                face_num = response.json()["result"]["face_num"]

                #置戴口罩标记为戴口罩了
                mask_valid = 1

                #获取人脸位置，画出图框
                for i in range(face_num):
                    mask_data = response.json()["result"]["face_list"][i]["mask"]["type"]

                    #发现没带口罩
                    if mask_data == 0:
                        mask_valid = 0

                if mask_valid == 0:        
                    #发现没带口罩，则播报提示
                    word_2_sound.trans_word_to_sound("请正确佩戴口罩",'tst_sound.wav')
                    os.system('mplayer ' + 'tst_sound.wav')
        
    except KeyboardInterrupt:
        os.remove('tst_sound.wav')
        print(task_name + "任务被终止")

if __name__ == "__main__":
    try:
        
        mydict=multiprocessing.Manager().dict()
        
        #　定义传递图像队列和传递图像处理结果队列
        q_frame = Queue()
        q_respond = Queue()
        
        #　采集摄像头进程、处理图片进程、播报语音信息
        get_camera_frame = Process(target=camera_frame_func, args=("获取摄像头图像", q_frame, mydict))
        proc_frame       = Process(target=proc_frame_func, args=("处理图像", q_frame, q_respond, mydict))
        read_rst         = Process(target=read_rst_func, args=("播报语音信息", q_respond))
              
        # 启动任务
        get_camera_frame.start()
        proc_frame.start()
        read_rst.start()

    except KeyboardInterrupt:
        os.remove('tst_sound.wav')
        os.remove('camera_pic.jpg')
        print("任务被终止了")

# 让我们临摹代码，学习“口罩”功能

In [ ]:
# #导入标准库
# import sys

# import os

# from playsound import playsound

# import cv2 as cv

# import time

# from multiprocessing import Process, Queue

# import multiprocessing


# #导入自定义库
# sys.path.append('../baidu_api_lib')

# from baidu_picture import baidu_picture_2_msg

# from baidu_sound import baidu_word_2_sound


# """ 公开课语音合成+人脸识别，可选更改为自己的api接口 """
# APP_ID = '21139072'

# API_KEY = 'tZ0j3YspNiGDhLeO7vxxvdzb'

# SECRET_KEY = 'UaaaMyO8RiZvXj0HxEt9DQavuk6u2uA3'


# #百度AI的调用url
# baidu_request_url = "https://aip.baidubce.com/rest/2.0/face/v3/detect"


# #处理图像的频率
# FRAME_PROC_FR = 80


# #　获取摄像头图像
# def camera_frame_func(task_name, mult_queue1, mydict):

    
#     # 创建一个VideoCapture对象
#     capture = cv.VideoCapture(0)   

    
#     #　给出提示信息
#     print(task_name + "任务启动")

    
#     #　传递视频的周期
#     pic_hz = 0

    
#     try:

#         while True:

#             # 一帧一帧读取视频
#             ret, frame = capture.read()

            
#             #　将图像发送至队列中
#             pic_hz+=1

#             if (pic_hz % FRAME_PROC_FR) == 1:   

#                 mult_queue1.put(frame) 

            
#             #获取人脸位置
#             if mydict:   

#                 #获取脸的个数
#                 face_num = mydict["result"]["face_num"]

                
#                 #有人脸才进行画框
#                 if face_num:

#                     #获取人脸位置，画出图框
#                     for i in range(face_num):

#                         location = mydict["result"]["face_list"][i]["location"]

#                         left_top = (int(location['left']), int(location['top']))

#                         right_bottom = (int(left_top[0] + location['width']), int(left_top[1] + location['height']))

#                         cv.rectangle(frame, left_top, right_bottom, (0,255,0),2) 

            
#             # 本地显示视频图像
#             cv.imshow('capture', frame)

#             cv.waitKey(1)


#     except KeyboardInterrupt:

#         # 释放cap,销毁窗口
#         capture.release()     

#         print(task_name + "任务被终止")

        
# #处理图像
# def proc_frame_func(task_name, mult_queue, mult_queue2, mydict):

#     #　给出提示信息
#     print(task_name + "任务启动")

    
#     # 传入百度AI的参数
#     pic_msg = baidu_picture_2_msg(APP_ID, API_KEY, SECRET_KEY)

    
#     try:

#         while True:

#             #从队列中获取图片
#             frame = mult_queue.get()

            
#             # 写入图片
#             cv.imwrite('camera_pic.jpg',frame) 

            
#             #从百度AI获取图片分析结果
#             response = pic_msg.pic_2_msg(baidu_request_url, 'camera_pic.jpg')

            
#             #获取分析数据是否正确
#             error_msg = response.json()["error_msg"]

            
#             #获取到正确信息
#             if error_msg == 'SUCCESS': 

#                 #更新人脸识别信息
#                 mydict["result"] = response.json()["result"]

            
#             #将播报信息进行对外发送
#             mult_queue2.put(response) 

            
#     except KeyboardInterrupt:

#         os.remove('camera_pic.jpg')

#         print(task_name + "任务被终止")

        
# #播报语音信息
# def read_rst_func(task_name, mult_queue):

#     print(task_name + "任务启动")

    
#     #传入百度AI的参数
#     word_2_sound = baidu_word_2_sound(APP_ID, API_KEY, SECRET_KEY)

    
#     try:

#         while True:

#             #从队列中获取播报信息
#             response = mult_queue.get()

            
#             #获取分析数据是否正确
#             error_msg = response.json()["error_msg"]

            
#             #获取到正确信息
#             if error_msg == 'SUCCESS':   

#                 #获取脸的个数
#                 face_num = response.json()["result"]["face_num"]


#                 #置戴口罩标记为戴口罩了
#                 mask_valid = 1


#                 #获取人脸位置，画出图框
#                 for i in range(face_num):

#                     mask_data = response.json()["result"]["face_list"][i]["mask"]["type"]


#                     #发现没带口罩
#                     if mask_data == 0:

#                         mask_valid = 0


#                 if mask_valid == 0:     

#                     #发现没带口罩，则播报提示
#                     word_2_sound.trans_word_to_sound("请正确佩戴口罩",'tst_sound.wav')

#                     os.system('mplayer ' + 'tst_sound.wav')

        
#     except KeyboardInterrupt:

#         os.remove('tst_sound.wav')

#         print(task_name + "任务被终止")


# if __name__ == "__main__":

#     try:

        
#         mydict=multiprocessing.Manager().dict()

        
#         #　定义传递图像队列和传递图像处理结果队列
#         q_frame = Queue()

#         q_respond = Queue()

        
#         #　采集摄像头进程、处理图片进程、播报语音信息
#         get_camera_frame = Process(target=camera_frame_func, args=("获取摄像头图像", q_frame, mydict))

#         proc_frame       = Process(target=proc_frame_func, args=("处理图像", q_frame, q_respond, mydict))

#         read_rst         = Process(target=read_rst_func, args=("播报语音信息", q_respond))

              
#         # 启动任务
#         get_camera_frame.start()

#         proc_frame.start()

#         read_rst.start()


#     except KeyboardInterrupt:

#         os.remove('tst_sound.wav')

#         os.remove('camera_pic.jpg')

#         print("任务被终止了")


# 课后练习
１、我们的人脸识别同时带有情绪检测，做一个情绪互动小游戏把。


In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#
# Copyright (c) 2020 Taste all Pi.
#
# Licensed under the GNU General Public License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#   http://www.gnu.org/licenses/gpl-2.0.html
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#导入标准库
import sys
import os
from playsound import playsound
import cv2 as cv
import time
from multiprocessing import Process, Queue
import multiprocessing
import random

#导入自定义库
sys.path.append('../baidu_api_lib')
from baidu_picture import baidu_picture_2_msg
from baidu_sound import baidu_word_2_sound

""" 公开课语音合成+人脸识别，可选更改为自己的api接口 """
APP_ID = '21139072'
API_KEY = 'tZ0j3YspNiGDhLeO7vxxvdzb'
SECRET_KEY = 'UaaaMyO8RiZvXj0HxEt9DQavuk6u2uA3'

#百度AI的调用url
baidu_request_url = "https://aip.baidubce.com/rest/2.0/face/v3/detect"

#处理图像的频率
FRAME_PROC_FR = 260

#情绪列表
emotion = {"angry":"愤怒", "disgust":"厌恶", "fear":"恐惧", "happy":"高兴", "sad":"伤心","surprise":"惊讶","neutral":"无表情", "pouty":"撅嘴", "grimace":"鬼脸"}

#　获取摄像头图像
def camera_frame_func(task_name, mult_queue1, mydict):
    
    # 创建一个VideoCapture对象
    capture = cv.VideoCapture(0)   
    
    #　给出提示信息
    print(task_name + "任务启动")
    
    #　传递视频的周期
    pic_hz = 0
    
    try:
        while True:
            # 一帧一帧读取视频
            ret, frame = capture.read()
            
            #　将图像发送至队列中
            pic_hz+=1
            if (pic_hz % FRAME_PROC_FR) == 1:     
                mult_queue1.put(frame) 
            
            #获取人脸位置
            if mydict:   
                #获取脸的个数
                face_num = mydict["result"]["face_num"]

                #获取人脸位置，画出图框
                for i in range(face_num):
                    location = mydict["result"]["face_list"][i]["location"]
                    left_top = (int(location['left']), int(location['top']))
                    right_bottom = (int(left_top[0] + location['width']), int(left_top[1] + location['height']))
                    cv.rectangle(frame, left_top, right_bottom, (0,255,0),2) 
            
            # 本地显示视频图像
            cv.imshow('capture', frame) 
            cv.waitKey(1)

    except KeyboardInterrupt:
        # 释放cap,销毁窗口
        capture.release()      
        print(task_name + "任务被终止")
        
#处理图像
def proc_frame_func(task_name, mult_queue, mult_queue2, mydict):
    #　给出提示信息
    print(task_name + "任务启动")
    
    # 传入百度AI的参数
    pic_msg = baidu_picture_2_msg(APP_ID, API_KEY, SECRET_KEY)
    
    try:
        while True:
            #从队列中获取图片
            frame = mult_queue.get()
            
            # 写入图片
            cv.imwrite('camera_pic.jpg',frame) 
            
            #从百度AI获取图片分析结果
            response = pic_msg.pic_2_msg(baidu_request_url, 'camera_pic.jpg')
            
            #获取分析数据是否正确
            error_msg = response.json()["error_msg"]
            
            #获取到正确信息
            if error_msg == 'SUCCESS':
                #更新人脸识别信息
                mydict["result"] = response.json()["result"]
            
            #将播报信息进行对外发送
            mult_queue2.put(response) 
            
    except KeyboardInterrupt:
        os.remove('camera_pic.jpg')
        print(task_name + "任务被终止")
        
#播报语音信息
def read_rst_func(task_name, mult_queue):
    print(task_name + "任务启动")
    
    #传入百度AI的参数
    word_2_sound = baidu_word_2_sound(APP_ID, API_KEY, SECRET_KEY)
    
    emotion_key = "happy"
    print(emotion[emotion_key])
    
    #发现没带口罩，则播报提示
    word_2_sound.trans_word_to_sound("我们做个游戏吧，我说表情，你来做",'tst_sound.mp3')
    os.system('mplayer ' + 'tst_sound.mp3')
    word_2_sound.trans_word_to_sound("请做出一个"+emotion[emotion_key]+"的表情",'tst_sound.mp3')
    os.system('mplayer ' + 'tst_sound.mp3')
    
    #表情做错次数
    emotion_err_times = 0
    
    try:
        while True:
            #从队列中获取播报信息
            response = mult_queue.get()
            
            #获取分析数据是否正确
            error_msg = response.json()["error_msg"]
            
            #获取到正确信息
            if error_msg == 'SUCCESS':    
                #获取脸的个数
                face_num = response.json()["result"]["face_num"]

                #获取表情信息，进行判断
                for i in range(face_num):
                    emotion_value = response.json()["result"]["face_list"][i]["emotion"]["type"]

                    #表情做的不对
                    if emotion_value != emotion_key:
                        word_2_sound.trans_word_to_sound("我要"+emotion[emotion_key]+",不是"+emotion[emotion_value]+"的表情",'tst_sound.wav')
                        os.system('mplayer ' + 'tst_sound.wav')
                        
                        #5次未作对，换一个表情
                        emotion_err_times += 1
                        if emotion_err_times >= 5:
                            #随机选择一个表情
                            for c in emotion.keys():
                                a = random.sample(emotion.keys(), 1)  
                                emotion_key = a[0]

                            word_2_sound.trans_word_to_sound("这个表情好难做啊,我们换一个"+emotion[emotion_key]+"的表情",'tst_sound.wav')
                            os.system('mplayer ' + 'tst_sound.wav')
                            
                            #清零做错次数
                            emotion_err_times = 0
                    #表情做对的了，换一个表情
                    else:     
                        #随机选择一个表情
                        for c in emotion.keys():
                            a = random.sample(emotion.keys(), 1)  
                            emotion_key = a[0]

                        word_2_sound.trans_word_to_sound("你做对了,再做一个"+emotion[emotion_key]+"的表情",'tst_sound.wav')
                        os.system('mplayer ' + 'tst_sound.wav')
                        
            else:
                word_2_sound.trans_word_to_sound("你去哪了，找不到你了",'tst_sound.wav')
                os.system('mplayer ' + 'tst_sound.wav')
                    
                    
    except KeyboardInterrupt:
        os.remove('tst_sound.wav')
        print(task_name + "任务被终止")

if __name__ == "__main__":
    try:
        
        mydict=multiprocessing.Manager().dict()
        
        #　定义传递图像队列和传递图像处理结果队列
        q_frame = Queue()
        q_respond = Queue()
        
        #　采集摄像头进程、处理图片进程、播报语音信息
        get_camera_frame = Process(target=camera_frame_func, args=("获取摄像头图像", q_frame, mydict))
        proc_frame       = Process(target=proc_frame_func, args=("处理图像", q_frame, q_respond, mydict))
        read_rst         = Process(target=read_rst_func, args=("播报语音信息", q_respond))
              
        # 启动任务
        get_camera_frame.start()
        proc_frame.start()
        read_rst.start()

    except KeyboardInterrupt:
        os.remove('tst_sound.wav')
        os.remove('camera_pic.jpg')
        print("任务被终止了")